# Szövegosztályozási feladat

Ahogy említettük, egy egyszerű szövegosztályozási feladatra fogunk koncentrálni az **AG_NEWS** adathalmazon alapulva, amelynek célja a hírcímek besorolása 4 kategória egyikébe: Világ, Sport, Üzlet és Tudomány/Technológia.

## Az adathalmaz

Ez az adathalmaz be van építve a [`torchtext`](https://github.com/pytorch/text) modulba, így könnyen hozzáférhetünk.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Itt a `train_dataset` és a `test_dataset` olyan gyűjteményeket tartalmaznak, amelyek párokat adnak vissza, például osztályszámot (címke) és szöveget.


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Szóval, nyomtassuk ki az adatállományunk első 10 új címsorát:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Mivel az adathalmazok iterátorok, ha többször szeretnénk használni az adatokat, listává kell alakítanunk:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenizálás

Most a szöveget **számokká** kell alakítanunk, amelyeket tenzorként lehet ábrázolni. Ha szószintű reprezentációt szeretnénk, két dolgot kell tennünk:
* használjunk egy **tokenizálót**, hogy a szöveget **tokenekre** bontsuk
* építsünk egy **szókincset** ezekből a tokenekből.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

A szókincs használatával könnyedén kódolhatjuk a tokenizált karakterláncot egy számhalmazzá:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Szavak zsákja szövegábrázolás

Mivel a szavak jelentést hordoznak, néha a szöveg jelentését pusztán az egyes szavak alapján is megérthetjük, függetlenül azok mondatbeli sorrendjétől. Például hírek osztályozásakor a *időjárás*, *hó* szavak valószínűleg *időjárás-előrejelzésre* utalnak, míg a *részvények*, *dollár* szavak inkább *pénzügyi hírekhez* kapcsolódnak.

A **Szavak zsákja** (BoW) vektorábrázolás a leggyakrabban használt hagyományos vektorábrázolás. Minden szó egy vektorindexhez van rendelve, a vektor elemei pedig azt mutatják, hogy egy adott dokumentumban hányszor fordul elő az adott szó.

![Kép, amely bemutatja, hogyan van ábrázolva a szavak zsákja vektor a memóriában.](../../../../../translated_images/hu/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Megjegyzés**: A BoW-t úgy is elképzelheted, mint az egyes szavak egy-egy one-hot-kódolt vektorának összegét a szövegben.

Az alábbiakban egy példa látható arra, hogyan lehet szavak zsákja ábrázolást generálni a Scikit Learn python könyvtár segítségével:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Az AG_NEWS adathalmaz vektoriális reprezentációjából származó bag-of-words vektor kiszámításához a következő függvényt használhatjuk:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Megjegyzés:** Itt a globális `vocab_size` változót használjuk a szókincs alapértelmezett méretének megadására. Mivel a szókincs mérete gyakran elég nagy, korlátozhatjuk a szókincs méretét a leggyakoribb szavakra. Próbáld meg csökkenteni a `vocab_size` értékét, és futtasd le az alábbi kódot, hogy lásd, hogyan befolyásolja ez a pontosságot. Néhány pontosságcsökkenésre számíthatsz, de nem drámaira, a nagyobb teljesítmény érdekében.


## BoW osztályozó tanítása

Most, hogy megtanultuk, hogyan készítsünk Bag-of-Words reprezentációt a szövegünkből, tanítsunk egy osztályozót erre az alapra. Először át kell alakítanunk az adatainkat úgy, hogy minden pozíciós vektorreprezentáció Bag-of-Words reprezentációvá legyen konvertálva. Ezt úgy érhetjük el, hogy a `bowify` függvényt `collate_fn` paraméterként adjuk meg a standard torch `DataLoader`-nek:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Most definiáljunk egy egyszerű osztályozó neurális hálózatot, amely egy lineáris réteget tartalmaz. Az input vektor mérete megegyezik a `vocab_size` értékével, az output mérete pedig az osztályok számával (4). Mivel osztályozási feladatot oldunk meg, a végső aktivációs függvény a `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Most már definiáljuk a standard PyTorch tanítási ciklust. Mivel az adatállományunk meglehetősen nagy, oktatási célból csak egy epochon keresztül fogunk tanítani, és néha még ennél is kevesebb ideig (az `epoch_size` paraméter megadásával korlátozhatjuk a tanítást). A tanítás során felhalmozott tanítási pontosságot is jelenteni fogjuk; a jelentés gyakoriságát a `report_freq` paraméterrel lehet megadni.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrammák, TriGrammák és N-Grammák

A szavak zsákja (bag of words) megközelítés egyik korlátja, hogy bizonyos szavak több szóból álló kifejezések részei lehetnek. Például a „hot dog” kifejezés teljesen más jelentéssel bír, mint a „hot” és „dog” szavak külön-külön más szövegkörnyezetben. Ha a „hot” és „dog” szavakat mindig ugyanazokkal a vektorokkal ábrázoljuk, az összezavarhatja a modellünket.

Ennek kezelésére gyakran használnak **N-gram reprezentációkat** dokumentumosztályozási módszerekben, ahol az egyes szavak, kétszavas vagy háromszavas kifejezések gyakorisága hasznos jellemző lehet az osztályozók tanításához. Például a bigram reprezentációban az eredeti szavakon túl az összes szópárt is hozzáadjuk a szókincshez.

Az alábbiakban egy példa látható arra, hogyan lehet bigram szavak zsákja reprezentációt létrehozni a Scikit Learn segítségével:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Az N-gram megközelítés fő hátránya, hogy a szókincs mérete rendkívül gyorsan növekedni kezd. Gyakorlatban az N-gram reprezentációt össze kell kombinálnunk valamilyen dimenziócsökkentési technikával, például *beágyazásokkal*, amelyeket a következő egységben fogunk tárgyalni.

Ahhoz, hogy az N-gram reprezentációt használjuk az **AG News** adatállományunkban, speciális ngram szókincset kell létrehoznunk:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


A fenti kódot használhatnánk a klasszifikátor betanítására, azonban ez nagyon memóriaigényes lenne. A következő egységben bigram klasszifikátort fogunk betanítani beágyazások segítségével.

> **Megjegyzés:** Csak azokat az ngramokat hagyhatod meg, amelyek a szövegben a megadott számnál többször fordulnak elő. Ez biztosítja, hogy a ritkán előforduló bigramok kimaradjanak, és jelentősen csökkenti a dimenziószámot. Ehhez állítsd a `min_freq` paramétert magasabb értékre, és figyeld meg a szókincs hosszának változását.


## Term Frekvencia és Invertált Dokumentum Frekvencia (TF-IDF)

A BoW (Bag of Words) reprezentációban a szavak előfordulásait egyenlő súllyal vesszük figyelembe, függetlenül a szó jelentőségétől. Azonban nyilvánvaló, hogy a gyakori szavak, mint például *a*, *az*, *és* stb., sokkal kevésbé fontosak az osztályozás szempontjából, mint a speciális kifejezések. Valójában a legtöbb NLP feladatban bizonyos szavak relevánsabbak, mint mások.

A **TF-IDF** a **term frekvencia–invertált dokumentum frekvencia** rövidítése. Ez a Bag of Words egy változata, ahol a bináris 0/1 érték helyett, amely egy szó megjelenését jelzi egy dokumentumban, egy lebegőpontos értéket használunk, amely a szó előfordulási gyakoriságával van összefüggésben a korpuszban.

Formálisabban, egy szó $i$ súlya egy dokumentumban $j$ az alábbiak szerint van meghatározva:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
ahol
* $tf_{ij}$ az $i$ szó előfordulásainak száma $j$-ben, vagyis a korábban látott BoW érték
* $N$ a gyűjteményben található dokumentumok száma
* $df_i$ azon dokumentumok száma, amelyek tartalmazzák az $i$ szót az egész gyűjteményben

A TF-IDF érték $w_{ij}$ arányosan növekszik azzal, hogy egy szó hányszor fordul elő egy dokumentumban, és csökken azzal, hogy hány dokumentumban található meg a korpuszban. Ez segít korrigálni azt a tényt, hogy bizonyos szavak gyakrabban fordulnak elő, mint mások. Például, ha egy szó *minden* dokumentumban megjelenik a gyűjteményben, akkor $df_i=N$, és $w_{ij}=0$, így ezek a kifejezések teljesen figyelmen kívül maradnak.

A Scikit Learn segítségével könnyedén létrehozhatunk TF-IDF vektorizációt szövegekhez:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Következtetés

Bár a TF-IDF reprezentációk súlyt adnak a különböző szavak gyakoriságának, nem képesek kifejezni a jelentést vagy a sorrendet. Ahogy a híres nyelvész, J. R. Firth 1935-ben mondta: „Egy szó teljes jelentése mindig kontextuális, és a jelentés tanulmányozása a kontextus figyelembevétele nélkül nem vehető komolyan.” A kurzus későbbi részében megtanuljuk, hogyan lehet a szövegből származó kontextuális információt megragadni nyelvi modellezés segítségével.



---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
